In [1]:
from setfit import SetFitModel, sample_dataset
import pandas as pd
from datasets import load_dataset
import numpy as np
import torch

/home/xiyan/data/conda/envs/GPU/lib/python3.11/site-packages/transformers/utils/generic.py:462: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/xiyan/data/conda/envs/GPU/lib/python3.11/site-packages/transformers/utils/generic.py:319: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


### Data preperation and model training

In [45]:
dataset = pd.read_csv("scratch/3_SML_benchmark.csv")

In [46]:
dataset.tail()

,Unnamed: 0,index,filename,text_original,text_clean,date,year,month,outlet,country,south,RA
475,475,101008,newsarticle/Toronto Star/Neutral & Urgent 2006...,\n Business\nTime to invest in foreign stude...,Business Time to invest in foreign students Da...,2022-09-10,2022,9,Toronto Star,Canada,0,0
476,476,100986,newsarticle/Toronto Star/Neutral & Urgent 2006...,\n Politics\nMeet the Greens who might take ...,Politics Meet the Greens who might take a run ...,2022-07-04,2022,7,Toronto Star,Canada,0,0
477,477,2962,A cautionary flag for oil development;Study ca...,"\nMarch 1, 2020 Sunday\n\n\nCopyright 2020 Tor...","March 1, 2020 Sunday Copyright 2020 Toronto St...",2020-03-01,2020,3,Toronto Star,Canada,0,0
478,478,74551,The Toronto Star_2018-8-68.txt,""""";""x""\n""1"";"" 82\n\n\n ...",""""";""x"" ""1"";"" 82 The Toronto Star August 5, 201...",2018-08-01,2018,8,Toronto Star,Canada,0,1
479,479,72479,The Toronto Star_2011-7-21_39703.txt,""""";""x""\n""1"";""DATE: 21.7.2011\nHEADLINE: Securi...",""""";""x"" ""1"";""DATE: 21.7.2011 HEADLINE: Security...",2011-07-01,2011,7,Toronto Star,Canada,0,1


In [47]:
df = dataset[["text_clean", "RA"]]
df.rename(columns={'RA': 'label','text_clean':'text'}, inplace=True)

/sctmp/xiyan/ipykernel_2848872/675981800.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'RA': 'label','text_clean':'text'}, inplace=True)


In [63]:
#train = df.sample(frac = 0.7)
train = df.groupby('label').sample(n=150)
df2 = df.drop(train.index)
validation = df2.sample(frac = 0.5)
test = df2.drop(validation.index)

In [64]:
train.to_csv("scratch/train.csv")
validation.to_csv("scratch/validation.csv")
test.to_csv("scratch/test.csv")

In [65]:
print(len(train),len(validation),len(test))

300 90 90


In [2]:
dataset = load_dataset("csv", data_files={'train':'scratch/train.csv','validation':'scratch/validation.csv','test': 'scratch/test.csv'})

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'label'],
        num_rows: 300
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'text', 'label'],
        num_rows: 90
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'label'],
        num_rows: 90
    })
})

In [4]:
#train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=50)
#train_dataset = dataset["train"]
eval_dataset = dataset["validation"]
test_dataset = dataset["test"]

In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define custom function for computing metrics
def compute_metrics(y_pred, y_test):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [9]:
torch.cuda.empty_cache()

In [10]:
from setfit import TrainingArguments

args = TrainingArguments(
    batch_size=16,
    num_epochs=1,
    max_length=512,
)

In [10]:
from setfit import Trainer

for i in [5, 10, 25, 50, 75, 100, 125, 150]:
    print(i)
    model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5") #original retrained model need to be loaded for each round
    train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=i)
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        metric=compute_metrics,
    )
    trainer.train()
    metrics = trainer.evaluate(eval_dataset)
    print(metrics)

5


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 60
  Batch size = 16
  Num epochs = 1
  Total optimization steps = 4


Step,Training Loss


...

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 45300
  Batch size = 16
  Num epochs = 1
  Total optimization steps = 2832


Step,Training Loss


***** Running evaluation *****


{'accuracy': 0.8222222222222222, 'precision': 0.8337889557135046, 'recall': 0.8069381598793364, 'f1': 0.812987012987013}


In [11]:
#recheck 100 sample and 150 sample again to see if the results above is subject to chance
from setfit import Trainer

model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5")
train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=100)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    metric=compute_metrics,
)
trainer.train()
metrics = trainer.evaluate(eval_dataset)
print(metrics)

/home/xiyan/data/conda/envs/GPU/lib/python3.11/site-packages/transformers/utils/generic.py:319: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 20200
  Batch size = 16
  Num epochs = 1
  Total optimization steps = 1263


Step,Training Loss


***** Running evaluation *****


{'accuracy': 0.8555555555555555, 'precision': 0.8984375, 'recall': 0.8333333333333333, 'f1': 0.8434782608695652}


In [10]:
from setfit import Trainer

model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5")
train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=150)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    metric=compute_metrics,
)
trainer.train()
metrics = trainer.evaluate(eval_dataset)
print(metrics)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 45300
  Batch size = 16
  Num epochs = 1
  Total optimization steps = 2832


Step,Training Loss


***** Running evaluation *****


{'accuracy': 0.8222222222222222, 'precision': 0.8337889557135046, 'recall': 0.8069381598793364, 'f1': 0.812987012987013}


**As N=100 sample works quite well and there's no huge improvements with introducing more samples, use N=100 sample for fine-tuning**

### Hyperparameter Optimization

In [5]:
from setfit import SetFitModel
from typing import Dict, Any

def model_init(params: Dict[str, Any]) -> SetFitModel:
    params = params or {}
    max_iter = params.get("max_iter", 100)
    solver = params.get("solver", "liblinear")
    params = {
        "head_params": {
            "max_iter": max_iter,
            "solver": solver,
        }
    }
    return SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5", **params)

In [6]:
from optuna import Trial
from typing import Dict, Union

def hp_space(trial: Trial) -> Dict[str, Union[float, int, str]]:
    return {
        "body_learning_rate": trial.suggest_float("body_learning_rate", 1e-5, 1e-3, log=True),
        "num_epochs": trial.suggest_int("num_epochs", 1, 4),
        "batch_size": trial.suggest_categorical("batch_size", [8, 16, 32, 64]),
        "seed": trial.suggest_int("seed", 1, 42),
        "max_iter": trial.suggest_int("max_iter", 50, 300),
        "solver": trial.suggest_categorical("solver", ["newton-cg", "lbfgs", "liblinear"]),
    }

In [12]:
#train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=100)

trainer = Trainer(
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    model_init=model_init,
    metric=compute_metrics,
)
best_run = trainer.hyperparameter_search(direction="maximize", hp_space=hp_space, n_trials=10)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

[I 2024-03-20 19:17:34,326] A new study created in memory with name: no-name-2c8f9552-2ca0-425f-b2ae-81c6859c64e6
Trial: {'body_learning_rate': 8.432222978081305e-05, 'num_epochs': 1, 'batch_size': 8, 'seed': 20, 'max_iter': 117, 'solver': 'lbfgs'}
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num unique pairs = 20200
  Batch size = 8
  Num epochs = 1
  Total optimization steps = 2525


Step,Training Loss


***** Running evaluation *****
[I 2024-03-20 19:24:10,476] Trial 0 finished with value: 3.366661234076094 and parameters: {'body_learning_rate': 8.432222978081305e-05, 'num_epochs': 1, 'batch_size': 8, 'seed': 20, 'max_iter': 117, 'solver': 'lbfgs'}. Best is trial 0 with value: 3.366661234076094.
Trial: {'body_learning_rate': 2.251620034128429e-05, 'num_epochs': 2, 'batch_size': 16, 'seed': 3, 'max_iter': 83, 'solver': 'liblinear'}
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num unique pairs = 20200
  Batch size = 16
  Num epochs = 2
  Total optimization steps = 2526


...

***** Running evaluation *****
/home/xiyan/data/conda/envs/GPU/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2024-03-20 21:07:55,653] Trial 7 finished with value: 1.4523255813953488 and parameters: {'body_learning_rate': 0.0009995390442812668, 'num_epochs': 3, 'batch_size': 16, 'seed': 9, 'max_iter': 141, 'solver': 'newton-cg'}. Best is trial 3 with value: 3.3687632393193128.
Trial: {'body_learning_rate': 1.455482691641014e-05, 'num_epochs': 3, 'batch_size': 16, 'seed': 2, 'max_iter': 84, 'solver': 'newton-cg'}
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num unique pairs =

Step,Training Loss


***** Running evaluation *****
[I 2024-03-20 21:26:50,597] Trial 8 finished with value: 3.238864262957134 and parameters: {'body_learning_rate': 1.455482691641014e-05, 'num_epochs': 3, 'batch_size': 16, 'seed': 2, 'max_iter': 84, 'solver': 'newton-cg'}. Best is trial 3 with value: 3.3687632393193128.
Trial: {'body_learning_rate': 0.0002175500130301556, 'num_epochs': 3, 'batch_size': 32, 'seed': 10, 'max_iter': 259, 'solver': 'lbfgs'}
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num unique pairs = 20200
  Batch size = 32
  Num epochs = 3
  Total optimization steps = 1896


Step,Training Loss


***** Running evaluation *****
[I 2024-03-20 21:44:42,411] Trial 9 finished with value: 3.0442697818536617 and parameters: {'body_learning_rate': 0.0002175500130301556, 'num_epochs': 3, 'batch_size': 32, 'seed': 10, 'max_iter': 259, 'solver': 'lbfgs'}. Best is trial 3 with value: 3.3687632393193128.


In [13]:
print(best_run)

BestRun(run_id='3', objective=3.3687632393193128, hyperparameters={'body_learning_rate': 5.757440332039611e-05, 'num_epochs': 2, 'batch_size': 32, 'seed': 9, 'max_iter': 278, 'solver': 'liblinear'}, backend=<optuna.study.study.Study object at 0x14a03c145fd0>)


In [14]:
trainer.apply_hyperparameters(best_run.hyperparameters, final_model=True)
trainer.train()

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num unique pairs = 20200
  Batch size = 32
  Num epochs = 2
  Total optimization steps = 1264


Step,Training Loss


In [15]:
trainer.evaluate(eval_dataset)

***** Running evaluation *****


{'accuracy': 0.8444444444444444,
 'precision': 0.8583925642427557,
 'recall': 0.8295625942684766,
 'f1': 0.8363636363636364}

In [16]:
trainer.evaluate(test_dataset)

***** Running evaluation *****


{'accuracy': 0.8666666666666667,
 'precision': 0.8805194805194805,
 'recall': 0.8624443344878774,
 'f1': 0.8642533936651584}

### Making predictions

In [17]:
df = pd.read_csv("scratch/2_data_preperation_data_dedup.csv")
df.drop(labels = ["Unnamed: 0"], axis = 1, inplace = True)

In [18]:
len(df)

214442

In [21]:
df.head()

,index,filename,text_original,year,month,outlet,country,text_clean,south
0,9029,"'Empire' may yet fade, as climate catastrophe ...","\r\nNovember 1, 2021\r\n\r\n\r\nCopyright 2021...",2021,11,The New Zealand Herald,New Zealand,"November 1, 2021 Copyright 2021 The New Zealan...",0
1,9030,'FIGHTING THE DARKNESS' New conservatism in th...,"\r\nAugust 22, 2020\r\n\r\n\r\nCopyright 2020 ...",2020,8,The New Zealand Herald,New Zealand,"August 22, 2020 Copyright 2020 The New Zealand...",0
2,9032,'Inspirational' leader heads home.rtf,"\r\nSeptember 4, 2021\r\n\r\n\r\nCopyright 202...",2021,9,The New Zealand Herald,New Zealand,"September 4, 2021 Copyright 2021 The New Zeala...",0
3,9034,'Native peoples hold key to climate solutions'...,"\r\nNovember 2, 2021\r\n\r\n\r\nCopyright 2021...",2021,11,The New Zealand Herald,New Zealand,"November 2, 2021 Copyright 2021 The New Zealan...",0
4,9037,"'Principled' woman stirs praise, tears.rtf","\r\nMarch 7, 2020\r\n\r\n\r\nCopyright 2020 Th...",2020,3,The New Zealand Herald,New Zealand,"March 7, 2020 Copyright 2020 The New Zealand H...",0


In [22]:
news_pred = df["text_clean"]

In [23]:
len(news_pred)

214442

In [24]:
preds = model.predict(news_pred)

In [25]:
df["keep"] = preds

In [26]:
df["keep"].value_counts()

keep
0    126451
1     87991
Name: count, dtype: int64

In [27]:
pre = df.loc[df["keep"]==1]

In [28]:
len(pre)

87991

In [29]:
pre.to_csv("scratch/3_fewshot_classification_data_final.csv")